In [1]:
import pandas as pd 
import requests
import requests
import os
import numpy as np
game_id = '0020500050'
def assist_paths(ps = False):
    carry='ps'
    if ps == False:
        carry=''
    for year in range(1997,2025):
        path = 'assists/'+str(year)+carry
        isExist = os.path.exists(path)
        if not isExist:
        
        # Create a new directory because it does not exist
            os.makedirs(path)
            print('Making Folder ' +path)
def pull_assists(game_id):
    url='https://stats.nba.com/stats/playbyplayv2?EndPeriod=0&GameID='+game_id+'&StartPeriod=0'

    headers = {
                                    "Host": "stats.nba.com",
                                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
                                    "Accept": "application/json, text/plain, */*",
                                    "Accept-Language": "en-US,en;q=0.5",
                                    "Accept-Encoding": "gzip, deflate, br",

                                    "Connection": "keep-alive",
                                    "Referer": "https://stats.nba.com/"
                                }
    jsond = requests.get(url,headers = headers).json()

    data= jsond['resultSets'][0]['rowSet']
 


    columns = jsond['resultSets'][0]['headers']
    df = pd.DataFrame.from_records(data, columns=columns)
    
    
    df1 =df.dropna(subset=['HOMEDESCRIPTION'])
    df1=df1[(df1.HOMEDESCRIPTION.str.lower().str.contains('ast'))]
    
    df2 =df.dropna(subset=['VISITORDESCRIPTION'])
    
    df2 = df2[df2.VISITORDESCRIPTION.str.lower().str.contains('ast')]
    
    test_df = pd.concat([df1,df2])

    test_df['PLAYER1_TEAM_ID']= test_df['PLAYER1_TEAM_ID'].astype(int)
  
    col = ['GAME_ID', 'EVENTNUM','PLAYER1_ID','PLAYER2_ID', 'PLAYER1_TEAM_ID']
    test_df= test_df[col]
    for c in col:
        test_df[col] = test_df[col].astype(str)
    test_df.columns =  ['GAME_ID', 'EVENTNUM','PLAYER_ID','ASSIST_ID', 'TEAM_ID']
    test_df['SHOT_ID'] = test_df['GAME_ID']+test_df['EVENTNUM']
    return test_df

def list_csv_files(path):
    csv_files = []
    for file in os.listdir(path):
        if file.endswith(".csv"):
            csv_files.append(file)
    return csv_files
def scrape_assists(ps = False):
    carry="ps"
    if ps == False:
        carry=""
    print('Starting')
    for year in range (1997,2025):
        print(year)
        path = "team/"+str(year)+carry
        
        files = list_csv_files(path)
        files = [f for f in files if 'vs' not in f]
        files = [f for f in files if 'avg' not in f]

        year_data = []
        teams=[]
        games = []
        for f in files:
            teamdf = pd.read_csv(path+'/'+f)
            teamdf['GAME_ID'] = teamdf['GAME_ID'].astype(str)
            games = games+ list(teamdf['GAME_ID'].unique())
            del teamdf
        games = list(set(games))

    
        assist_path = 'assists/'+str(year)+carry+'/ast.csv'
        if os.path.exists(assist_path):
            print('Exists')
            print(len(games))
            logged = pd.read_csv(assist_path)
            logged['GAME_ID'] = logged['GAME_ID'].astype(str)
            logged_games = list(logged['GAME_ID'].unique())
            print(len(logged_games))
            games = [game for game in games if game not in logged_games]
            
        print(len(games))
        
        game_count = 0
        if game_count % 25 == 0:
            print('test')
           
        for game in games:
            game = str(game)
            if game[0:2] !='00':
                game = '00'+game
            df = pull_assists(game)
       
            year_data.append(df)
            game_count+=1

            if game_count %50 == 0:
                if os.path.exists(assist_path):
                    old_frame = pd.read_csv(assist_path)
                    new_frame = pd.concat(year_data)
    
                    to_save = pd.concat([old_frame,new_frame])
                    to_save.to_csv(assist_path,index = False)
                    year_data = []
    
                    print('50 hit, saving')
                else:
                    to_save = pd.concat(year_data)
                    to_save.to_csv(assist_path)
                    year_data= []
                    print('50 hit, saving')
        if len(games)  >0:          
            old_frame = pd.read_csv(assist_path)
            new_frame = pd.concat(year_data)
    
            to_save = pd.concat([old_frame,new_frame])
            to_save.to_csv(assist_path,index = False)
    
            print(' Year done ' +str(year)+carry)
        else:
            print('already scraped')
            print('Hit')
#assist_paths(ps=False)
#assist_paths(ps=True)
scrape_assists(ps=True)


Starting
1997
Exists
72
72
0
test
already scraped
Hit
1998
Exists
71
71
0
test
already scraped
Hit
1999
Exists
66
66
0
test
already scraped
Hit
2000
Exists
75
75
0
test
already scraped
Hit
2001
Exists
71
71
0
test
already scraped
Hit
2002
Exists
71
71
0
test
already scraped
Hit
2003
Exists
88
88
0
test
already scraped
Hit
2004
Exists
82
82
0
test
already scraped
Hit
2005
Exists
84
84
0
test
already scraped
Hit
2006
Exists
89
89
0
test
already scraped
Hit
2007
Exists
79
79
0
test
already scraped
Hit
2008
Exists
86
86
0
test
already scraped
Hit
2009
Exists
85
85
0
test
already scraped
Hit
2010
Exists
82
82
0
test
already scraped
Hit
2011
Exists
81
81
0
test
already scraped
Hit
2012
Exists
84
84
0
test
already scraped
Hit
2013
Exists
85
85
0
test
already scraped
Hit
2014
Exists
89
89
0
test
already scraped
Hit
2015
Exists
81
81
0
test
already scraped
Hit
2016
Exists
86
86
0
test
already scraped
Hit
2017
Exists
79
79
0
test
already scraped
Hit
2018
Exists
82
82
0
test
already scraped
Hit
2

In [2]:
def clean_assists(ps = False):
    carry="ps"
    if ps == False:
        carry=""

    for year in range (1997,2025):
  
        path = "assists/"+str(year)+carry+"/ast.csv"
        df = pd.read_csv(path)
        if 'Unnamed: 0' in df.columns:
            df.drop(columns=['Unnamed: 0'],inplace=True)
        df.to_csv(path,index=False)
clean_assists()
clean_assists(ps= True)

In [3]:
year = 2004
path = "team/"+str(year)

files = list_csv_files(path)
files = [f for f in files if 'vs' in f]
frames = []
for f in files:
    file = path+'/'+f
    df = pd.read_csv(file)
    frames.append(df)
df = pd.concat(frames)
oldgames = list(df['GAME_ID'].unique())


In [4]:
assist_df= pd.read_csv("assists/"+str(year)+"/ast.csv")
assist_df['GAME_ID'] = assist_df['GAME_ID'].astype(str)
assist_games = list(assist_df['GAME_ID'].unique())